# Parking Garage Project

Your assignment for today is to create a parking garage class to get more familiar with Object Oriented Programming(OOP). 

Your parking gargage class should have the following methods:
- takeTicket
   - This should decrease the amount of tickets available by 1
   - This should decrease the amount of parkingSpaces available by 1
- payForParking
   - Display an input that waits for an amount from the user and store it in a variable
   - If the payment variable is not empty then ->  display a message to the user that their ticket has been paid and they have 15mins to leave
   - This should update the "currentTicket" dictionary key "paid" to True
- leaveGarage
   - If the ticket has been paid, display a message of "Thank You, have a nice day"
   - If the ticket has not been paid, display an input prompt for payment
      - Once paid, display message "Thank you, have a nice day!"
   - Update parkingSpaces list to increase by 1
   - Update tickets list to increase by 1

You will need a few attributes as well:
- tickets -> list
- parkingSpaces -> list
- currentTicket -> dictionary

In [ ]:
#Start your code here...

from time import perf_counter
import sqlite3
from datetime import datetime


class parkingGarage:
    def __init__(self, parkingSpaces, rate, gracePeriod):
        self.rate = rate # per minute
        self.gracePeriod = gracePeriod # in seconds
        self.tickets = {}
        self.parkingSpaces = list(range(1,parkingSpaces+1))
        self.parkTimes = {}
        self.leaveTimes = {}
        
    def receipt(self, t, amt, elapsed_time):
        
        # Offer and, if prompted, create receipt with date and time
        now = datetime.now()
        year = now.strftime("%Y")
        month = now.strftime("%m")
        day = now.strftime("%d")
        time = now.strftime("%H:%M:%S")
        #date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
        
        amt = float(amt)
        elapsed_time = "%.2f" % round(elapsed_time, 2)
        
        receipt = [t, now, elapsed_time, amt]
        for i in receipt:
            print(i)
        for i in receipt:
            print(type(i))

        # Store parking data using datetime and sqlite3
        conn = sqlite3.connect('parking_data.sqlite')
        cur = conn.cursor()
        
        cur.execute('INSERT INTO Parks (ticket, datetime, duration, amount) VALUES (?, ?, ?, ?)', (t, now, elapsed_time, amt))
 
    def takeTicket(self):
        
        # Remove next available space from parkingSpaces and add it to tickets. If no available spaces,
        # notify user.
        if self.parkingSpaces == []:
            print('Sorry, the garage is full. Next time ride a bike or take the bus! Have a nice day.')
        
        else:
            new_park = self.parkingSpaces.pop(0)
            print(f'Your ticket number is {new_park}.')
            self.tickets[new_park] = 'unpaid'
            
            # Store current time in parkTimes
            self.parkTimes[new_park] = perf_counter()
            
    def showCharges(self):
        
        # Prompt for ticket number and calculate parking time
        ticket = 0
        try:
            ticket = int(input('Please enter your ticket number.'))
        except:
            print('Entry not recognized.')
                    
        if ticket not in self.tickets.keys():
            print('Ticket not found. Please retry.')
        
        else:
            park_time = perf_counter() - self.parkTimes[ticket]
        
            # Calculate amount due with rate and display current amount owed
            amt_due = (park_time / 60) * self.rate
            amt_due_round = "%.2f" % round(amt_due, 2)
            print(f'Amount due: ${amt_due_round}.')
                    
    def payForParking(self):
        
        # Prompt for ticket number and calculate parking time
        ticket = 0
        try:
            ticket = int(input('Please enter your ticket number.'))
        except:
            print('Entry not recognized.')
                    
        if ticket not in self.tickets.keys():
            print('Ticket not found. Please retry.')
        
        else:
            park_time = perf_counter() - self.parkTimes[ticket]
        
            # Calculate amount due with rate and ask for payment
            amt_due = (park_time / 60) * self.rate
            amt_due_round = "%.2f" % round(amt_due, 2)
            payment = input(f'Amount due: ${amt_due_round}. Press any key to pay.')
        
            if payment:
                self.tickets[ticket] = amt_due_round
                print('self.tickets[ticket]: ', self.tickets[ticket])
                grace_mins = int(self.gracePeriod / 60)
                print(f'Thank you for your payment. You have {grace_mins} minutes to leave the garage.')                
            
                # Delete park counter for ticket
                del self.parkTimes[ticket]
            
                # Start counter for leave window and store in leaveTimes
                self.leaveTimes[ticket] = perf_counter()                                       

    def leaveGarage(self):
        
        # Prompt user for ticket number and determine payment status
        verify = 0        
        try:
            verify = int(input('Please enter your ticket number.'))
        except:
            print('Entry not recognized.')
        
        if verify not in self.tickets.keys():
            print('Ticket not found. Please retry.')
        
        elif self.tickets[verify] == 'unpaid':
            print('Payment is required to exit garage. Enter "pay" at prompt to pay and exit.')
        
        else:
            print('self.tickets[verify]: ', self.tickets[verify])
            elapsed_time = perf_counter() - self.leaveTimes[verify]
            test = isinstance(self.tickets[verify], float)
            print(type(self.tickets[verify]))
            
            if elapsed_time <= self.gracePeriod and test != 'unpaid':
                
                # Remove k,v from dict and append k to spaces list
                self.parkingSpaces.append(verify)
                removed_ticket = self.tickets.pop(verify, 'No ticket found')
                print(removed_ticket)
                
                receipt_query = input('Would you like a receipt? y/n')
                if receipt_query.lower() == 'y':
                    print('v:', type(verify))
                    print('ev:', type(elapsed_time))
                    self.receipt(verify, removed_ticket, elapsed_time)
                    
                else:
            
                    # Delete leave counter for ticket
                    del self.leaveTimes[verify]
                    print('Thank you. Please exit slowly and drive safely.')
        
            elif elapsed_time > self.gracePeriod:
            
                # Calculate and charge for time over the leave window
                elapsed_time = perf_counter() - self.leaveTimes[verify] - self.gracePeriod
                amount_due = (elapsed_time / 60) * self.rate
                amount_due_round = "%.2f" % round(amount_due, 2)
                payment = input(
                    f'The time for leaving has expired. Please pay: ${amount_due_round}. Press any key to pay.'
                    )
            
                if payment:
                
                    # Remove k,v from dict and append k to spaces list
                    self.parkingSpaces.append(verify)
                    removed_ticket = self.tickets.pop(verify, 'No ticket found')
                    
                    receipt_query = input('Would you like a receipt? y/n')
                    if receipt_query.lower() == 'y':
                        self.receipt(verify, removed_ticket, elapsed_time)
                    
                    else:
            
                        # Delete leave counter for ticket
                        del self.leaveTimes[verify]
                        print('Thank you. Please exit slowly and drive safely.')
    
    
                                
def run():
    garage = parkingGarage(10,1,600)
    while True:
        response = input('What would you like to do: pay, park, show charges, or leave?')
        if response.lower() == 'park':
            garage.takeTicket()
        elif response.lower() == 'show charges':
            garage.showCharges()
        elif response.lower() == 'pay':
            garage.payForParking()
        elif response.lower() == 'leave':
            garage.leaveGarage()
            #garage.recordParks()
        elif response.lower() == 'quit':
            break
            
run()

In [34]:
import sqlite3
conn = sqlite3.connect('parking_data.sqlite')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS Parks')
#cur.execute('CREATE TABLE Parks (ticket INT, date STR, time STR, duration FLOAT, amount FLOAT)')
cur.execute('CREATE TABLE Parks (ticket INTEGER, datetime TEXT, duration NUMERIC, amount NUMERIC)')

conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect('parking_data.sqlite')
cur = conn.cursor()

print('Parking data:')
cur.execute('SELECT ticket, datetime, duration, amount FROM Parks')
for row in cur:
    print(row)

